In [1]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot
import matplotlib.pyplot as plt
from keras.utils import plot_model
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
# set seed to reproduce similar results
np.random.seed(0)

2023-09-29 05:31:37.281917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!mv /Users/mangodingo/Desktop/VS/Advanced AI/0922/human_activity human_activity

mv: rename /Users/mangodingo/Desktop/VS/Advanced to human_activity/Advanced: No such file or directory
mv: rename AI/0922/human_activity to human_activity/human_activity: No such file or directory


In [3]:

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                         if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv('/Users/mangodingo/Desktop/VS/Advanced_AI/0922/human_activity/features.txt',sep='\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name )
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None,names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt',sep='\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 get_human_dataset( )을 이용해 학습/테스트용 DataFrame 반환
X_train, X_test, y_train, y_test = get_human_dataset()

print("## 학습 피처 데이터 정보 ##")
print(X_train.info())

## 학습 피처 데이터 정보 ##
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
# 랜덤 포레스트

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train , y_train)

pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test , pred_rf)

print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_rf))
print("**** Confusion Matrix ****")
print(confusion_matrix(y_test, pred_rf))
print("**** Classification Report ****")
print(classification_report(y_test, pred_rf))

랜덤 포레스트 정확도: 0.9253
**** Confusion Matrix ****
[[486   4   6   0   0   0]
 [ 38 426   7   0   0   0]
 [ 22  44 354   0   0   0]
 [  0   0   0 440  51   0]
 [  0   0   0  48 484   0]
 [  0   0   0   0   0 537]]
**** Classification Report ****
              precision    recall  f1-score   support

           1       0.89      0.98      0.93       496
           2       0.90      0.90      0.90       471
           3       0.96      0.84      0.90       420
           4       0.90      0.90      0.90       491
           5       0.90      0.91      0.91       532
           6       1.00      1.00      1.00       537

    accuracy                           0.93      2947
   macro avg       0.93      0.92      0.92      2947
weighted avg       0.93      0.93      0.93      2947



In [15]:
# 로지스틱 회귀
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)

pred_lr = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test , pred_lr)

print('로지스틱 회귀 정확도: {0:.4f}'.format(accuracy_lr))
print("**** Confusion Matrix ****")
print(confusion_matrix(y_test, pred_lr))
print("**** Classification Report ****")
print(classification_report(y_test, pred_lr))

로지스틱 회귀 정확도: 0.9579
**** Confusion Matrix ****
[[492   1   3   0   0   0]
 [ 24 445   2   0   0   0]
 [  4  14 402   0   0   0]
 [  0   3   0 431  57   0]
 [  0   0   0  16 516   0]
 [  0   0   0   0   0 537]]
**** Classification Report ****
              precision    recall  f1-score   support

           1       0.95      0.99      0.97       496
           2       0.96      0.94      0.95       471
           3       0.99      0.96      0.97       420
           4       0.96      0.88      0.92       491
           5       0.90      0.97      0.93       532
           6       1.00      1.00      1.00       537

    accuracy                           0.96      2947
   macro avg       0.96      0.96      0.96      2947
weighted avg       0.96      0.96      0.96      2947



In [16]:
# 결정트리
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)

pred_dt = dt.predict(X_test)
accuracy_dt = accuracy_score(y_test , pred_dt)

print('결정트리 정확도: {0:.4f}'.format(accuracy_dt))
print("**** Confusion Matrix ****")
print(confusion_matrix(y_test, pred_dt))
print("**** Classification Report ****")
print(classification_report(y_test, pred_dt))

결정트리 정확도: 0.8595
**** Confusion Matrix ****
[[448  24  24   0   0   0]
 [ 74 367  30   0   0   0]
 [ 23  46 351   0   0   0]
 [  0   0   0 373 118   0]
 [  0   0   0  75 457   0]
 [  0   0   0   0   0 537]]
**** Classification Report ****
              precision    recall  f1-score   support

           1       0.82      0.90      0.86       496
           2       0.84      0.78      0.81       471
           3       0.87      0.84      0.85       420
           4       0.83      0.76      0.79       491
           5       0.79      0.86      0.83       532
           6       1.00      1.00      1.00       537

    accuracy                           0.86      2947
   macro avg       0.86      0.86      0.86      2947
weighted avg       0.86      0.86      0.86      2947



In [17]:
# 서포트 벡터 머신
from sklearn.svm import SVC

svm = SVC(random_state=0)
svm.fit(X_train, y_train)

pred_svm = svm.predict(X_test)
accuracy_svm = accuracy_score(y_test , pred_svm)

print('서포트 벡터 머신 정확도: {0:.4f}'.format(accuracy_svm))
print("**** Confusion Matrix ****")
print(confusion_matrix(y_test, pred_svm))
print("**** Classification Report ****")
print(classification_report(y_test, pred_svm))

서포트 벡터 머신 정확도: 0.9505
**** Confusion Matrix ****
[[488   5   3   0   0   0]
 [ 20 451   0   0   0   0]
 [ 10  26 384   0   0   0]
 [  0   2   0 438  51   0]
 [  0   0   0  29 503   0]
 [  0   0   0   0   0 537]]
**** Classification Report ****
              precision    recall  f1-score   support

           1       0.94      0.98      0.96       496
           2       0.93      0.96      0.94       471
           3       0.99      0.91      0.95       420
           4       0.94      0.89      0.91       491
           5       0.91      0.95      0.93       532
           6       1.00      1.00      1.00       537

    accuracy                           0.95      2947
   macro avg       0.95      0.95      0.95      2947
weighted avg       0.95      0.95      0.95      2947



In [19]:
# linearSVM

from sklearn.svm import LinearSVC

lsvm = LinearSVC(random_state=0)
lsvm.fit(X_train, y_train)

pred_lsvm = lsvm.predict(X_test)
accuracy_lsvm = accuracy_score(y_test , pred_svm)

print('선형 서포트벡터 머신 정확도: {0:.4f}'.format(accuracy_lsvm))
print("**** Confusion Matrix ****")
print(confusion_matrix(y_test, pred_lsvm))
print("**** Classification Report ****")
print(classification_report(y_test, pred_lsvm))

선형 서포트벡터 머신 정확도: 0.9505
**** Confusion Matrix ****
[[496   0   0   0   0   0]
 [ 17 454   0   0   0   0]
 [  2   6 412   0   0   0]
 [  0   3   0 429  57   2]
 [  1   0   0  12 519   0]
 [  0   0   0   0   0 537]]
**** Classification Report ****
              precision    recall  f1-score   support

           1       0.96      1.00      0.98       496
           2       0.98      0.96      0.97       471
           3       1.00      0.98      0.99       420
           4       0.97      0.87      0.92       491
           5       0.90      0.98      0.94       532
           6       1.00      1.00      1.00       537

    accuracy                           0.97      2947
   macro avg       0.97      0.97      0.97      2947
weighted avg       0.97      0.97      0.97      2947

